<a href="https://colab.research.google.com/github/Ad2891/Slutuppgift-team12/blob/main/Slutuppgift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Slutuppgift 1 - Identifying Employee Attrition 

## The Problem Question

During this task, it is our goal to identify features of interest that can be used to predict the reasons for employee attrition. So the question used for this problem is:

>**How can prior indicators from features be used to predict if an employee is at risk of leaving the company?**



In [15]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn

from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report

First we retrieve the data.

In [16]:
#Import .csv file and save in variable employee_data 
employee_data = pd.read_csv("/content/WA_Fn-UseC_-HR-Employee-Attrition.csv", sep=",")
employee_data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [17]:
employee_data_attrition_dummy = pd.get_dummies(employee_data, columns=['Attrition'], drop_first=True) 

In [18]:
employee_data_attrition_gender_dummy = pd.get_dummies(employee_data, columns=['Gender'], drop_first=True) 

In [21]:
employee_data_attrition_gender_business_dummy = employee_data_attrition_gender_dummy
employee_data_attrition_gender_business_dummy.loc[
                employee_data_attrition_gender_business_dummy['BusinessTravel'] 
                == 'Travel_Rarely', 'BusinessTravel'] = 3
employee_data_attrition_gender_business_dummy.loc[
                employee_data_attrition_gender_business_dummy['BusinessTravel'] 
                == 'Travel_Frequently', 'BusinessTravel'] = 2
employee_data_attrition_gender_business_dummy.loc[
                employee_data_attrition_gender_business_dummy['BusinessTravel'] 
                == 'No_Travel', 'BusinessTravel'] = 1

In [22]:
employee_data_attrition_gender_business_dummy.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Gender_Male
0,41,Yes,3,1102,Sales,1,2,Life Sciences,1,1,...,80,0,8,0,1,6,4,0,5,0
1,49,No,2,279,Research & Development,8,1,Life Sciences,1,2,...,80,1,10,3,3,10,7,1,7,1
2,37,Yes,3,1373,Research & Development,2,2,Other,1,4,...,80,0,7,3,3,0,0,0,0,1
3,33,No,2,1392,Research & Development,3,4,Life Sciences,1,5,...,80,0,8,3,3,8,7,3,0,0
4,27,No,3,591,Research & Development,2,1,Medical,1,7,...,80,1,6,3,3,2,2,2,2,1


In [23]:
employee_data_attrition_gender_business_dep_dummy = employee_data_attrition_gender_business_dummy
employee_data_attrition_gender_business_dep_dummy.loc[
                employee_data_attrition_gender_business_dep_dummy['Department'] 
                == 'Sales', 'Department'] = 3
employee_data_attrition_gender_business_dep_dummy.loc[
                employee_data_attrition_gender_business_dep_dummy['Department'] 
                == 'Research & Development', 'Department'] = 2
employee_data_attrition_gender_business_dep_dummy.loc[
                employee_data_attrition_gender_business_dep_dummy['Department'] 
                == 'Human Resources', 'Department'] = 1

In [24]:
employee_data_attrition_gender_business_dep_dummy.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Gender_Male
0,41,Yes,3,1102,3,1,2,Life Sciences,1,1,...,80,0,8,0,1,6,4,0,5,0
1,49,No,2,279,2,8,1,Life Sciences,1,2,...,80,1,10,3,3,10,7,1,7,1
2,37,Yes,3,1373,2,2,2,Other,1,4,...,80,0,7,3,3,0,0,0,0,1
3,33,No,2,1392,2,3,4,Life Sciences,1,5,...,80,0,8,3,3,8,7,3,0,0
4,27,No,3,591,2,2,1,Medical,1,7,...,80,1,6,3,3,2,2,2,2,1


In [25]:
employee_data_attrition_gender_business_dep_educ_dummy = employee_data_attrition_gender_business_dep_dummy
employee_data_attrition_gender_business_dep_educ_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_dummy['EducationField'] 
                == 'Technical Degree', 'EducationField'] = 6
employee_data_attrition_gender_business_dep_educ_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_dummy['EducationField'] 
                == 'Other', 'EducationField'] = 5
employee_data_attrition_gender_business_dep_educ_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_dummy['EducationField'] 
                == 'Medical', 'EducationField'] = 4
employee_data_attrition_gender_business_dep_educ_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_dummy['EducationField'] 
                == 'Marketing', 'EducationField'] = 3
employee_data_attrition_gender_business_dep_educ_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_dummy['EducationField'] 
                == 'Life Sciences', 'EducationField'] = 2
employee_data_attrition_gender_business_dep_educ_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_dummy['EducationField'] 
                == 'Human Resources', 'EducationField'] = 1

In [26]:
employee_data_attrition_gender_business_dep_educ_dummy.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Gender_Male
0,41,Yes,3,1102,3,1,2,2,1,1,...,80,0,8,0,1,6,4,0,5,0
1,49,No,2,279,2,8,1,2,1,2,...,80,1,10,3,3,10,7,1,7,1
2,37,Yes,3,1373,2,2,2,5,1,4,...,80,0,7,3,3,0,0,0,0,1
3,33,No,2,1392,2,3,4,2,1,5,...,80,0,8,3,3,8,7,3,0,0
4,27,No,3,591,2,2,1,4,1,7,...,80,1,6,3,3,2,2,2,2,1


In [29]:
employee_data_attrition_gender_business_dep_educ_job_dummy = employee_data_attrition_gender_business_dep_educ_dummy
employee_data_attrition_gender_business_dep_educ_job_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_dummy['JobRole'] 
                == 'Sales Representative', 'JobRole'] = 9
employee_data_attrition_gender_business_dep_educ_job_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_dummy['JobRole'] 
                == 'Sales Executive', 'JobRole'] = 8
employee_data_attrition_gender_business_dep_educ_job_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_dummy['JobRole'] 
                == 'Research Scientist', 'JobRole'] = 7
employee_data_attrition_gender_business_dep_educ_job_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_dummy['JobRole'] 
                == 'Research Director', 'JobRole'] = 6
employee_data_attrition_gender_business_dep_educ_job_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_dummy['JobRole'] 
                == 'Manufacturing Director', 'JobRole'] = 5
employee_data_attrition_gender_business_dep_educ_job_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_dummy['JobRole'] 
                == 'Manager', 'JobRole'] = 4
employee_data_attrition_gender_business_dep_educ_job_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_dummy['JobRole'] 
                == 'Laboratory Technician', 'JobRole'] = 3
employee_data_attrition_gender_business_dep_educ_job_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_dummy['JobRole'] 
                == 'Human Resources', 'JobRole'] = 2
employee_data_attrition_gender_business_dep_educ_job_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_dummy['JobRole'] 
                == 'Healthcare Representative', 'JobRole'] = 1

In [30]:
employee_data_attrition_gender_business_dep_educ_job_dummy.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Gender_Male
0,41,Yes,3,1102,3,1,2,2,1,1,...,80,0,8,0,1,6,4,0,5,0
1,49,No,2,279,2,8,1,2,1,2,...,80,1,10,3,3,10,7,1,7,1
2,37,Yes,3,1373,2,2,2,5,1,4,...,80,0,7,3,3,0,0,0,0,1
3,33,No,2,1392,2,3,4,2,1,5,...,80,0,8,3,3,8,7,3,0,0
4,27,No,3,591,2,2,1,4,1,7,...,80,1,6,3,3,2,2,2,2,1


In [31]:
employee_data_attrition_gender_business_dep_educ_job_rel_dummy = employee_data_attrition_gender_business_dep_educ_job_dummy
employee_data_attrition_gender_business_dep_educ_job_rel_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_rel_dummy['MaritalStatus'] 
                == 'Single', 'MaritalStatus'] = 3
employee_data_attrition_gender_business_dep_educ_job_rel_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_rel_dummy['MaritalStatus'] 
                == 'Married', 'MaritalStatus'] = 2
employee_data_attrition_gender_business_dep_educ_job_rel_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_rel_dummy['MaritalStatus'] 
                == 'Divorced', 'MaritalStatus'] = 1

In [32]:
employee_data_attrition_gender_business_dep_educ_job_rel_over_dummy = employee_data_attrition_gender_business_dep_educ_job_rel_dummy
employee_data_attrition_gender_business_dep_educ_job_rel_over_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_rel_over_dummy['Over18'] 
                == 'Y', 'Over18'] = 1
employee_data_attrition_gender_business_dep_educ_job_rel_over_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_rel_over_dummy['Over18'] 
                == 'N', 'Over18'] = 0

In [33]:
employee_data_attrition_gender_business_dep_educ_job_rel_over_time_dummy = employee_data_attrition_gender_business_dep_educ_job_rel_over_dummy
employee_data_attrition_gender_business_dep_educ_job_rel_over_time_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_rel_over_time_dummy['OverTime'] 
                == 'Yes', 'OverTime'] = 1
employee_data_attrition_gender_business_dep_educ_job_rel_over_time_dummy.loc[
                employee_data_attrition_gender_business_dep_educ_job_rel_over_time_dummy['OverTime'] 
                == 'No', 'OverTime'] = 0

In [34]:
employee_data_dummies = employee_data_attrition_gender_business_dep_educ_job_rel_over_time_dummy
employee_data_dummies.loc[
                employee_data_dummies['Attrition'] 
                == 'Yes', 'Attrition'] = 1
employee_data_dummies.loc[
                employee_data_dummies['Attrition'] 
                == 'No', 'Attrition'] = 0

In [35]:
employee_data_dummies.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Gender_Male
0,41,1,3,1102,3,1,2,2,1,1,...,80,0,8,0,1,6,4,0,5,0
1,49,0,2,279,2,8,1,2,1,2,...,80,1,10,3,3,10,7,1,7,1
2,37,1,3,1373,2,2,2,5,1,4,...,80,0,7,3,3,0,0,0,0,1
3,33,0,2,1392,2,3,4,2,1,5,...,80,0,8,3,3,8,7,3,0,0
4,27,0,3,591,2,2,1,4,1,7,...,80,1,6,3,3,2,2,2,2,1
